### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

In [3]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### GOOGLE SHEETS

In [4]:
# Descargo los filtros de Campañas
fc = descarga('1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw','Filtrar')
# Mando a mayusculas
fc[fc.columns[0]] = fc[fc.columns[0]].str.upper()
fc = fc[fc.columns[0]].tolist()

Descarga Correcta!


### FUNCIONES

In [5]:
def filtrar_cam(i):
    val = 'No'
    for j in fc:
        if j in i:
            val = 'Si'
            break
    return val        

### QUERIES

In [6]:
# 60GB
q_tc = '''WITH coupons_table AS (
    SELECT cn.country_name AS country,
           IFNULL(c.city_name,'-') AS city,
           tc.talon_campaign_id AS campaign_id,
           tc.coupon_id AS coupon_id,
           o.registered_date AS fecha,
           COUNT(DISTINCT o.order_id) AS redeemed,
           COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS redeemed_confirmed,
           COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS acquisitions,
           --SUM(CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN tc.coupon_used_amount ELSE 0 END) AS used_amount,
           --SUM(CASE WHEN bi.payment_mode != 'TOTAL_AMOUNT' THEN tc.coupon_used_amount ELSE 0 END) AS used_amount_shopper,
           --SUM(CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS used_amount_usd,
           --SUM(CASE WHEN bi.payment_mode != 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS used_amount_usd_shopper,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' AND bi.payment_mode = 'TOTAL_AMOUNT' THEN tc.coupon_used_amount ELSE 0 END) AS used_amount_confirmed,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' AND bi.payment_mode != 'TOTAL_AMOUNT' THEN tc.coupon_used_amount ELSE 0 END) AS used_amount_confirmed_shopper,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' AND bi.payment_mode = 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS used_amount_confirmed_usd,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' AND bi.payment_mode != 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS used_amount_confirmed_usd_shopper
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    INNER JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON tc.coupon_country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
    LEFT JOIN `peya-argentina.user_santiago_curat.city_usuarios` AS u ON tc.user_id = u.user AND cn.country_name = u.country
    WHERE o.registered_date BETWEEN DATE_ADD(CURRENT_DATE(),INTERVAL -30 DAY) AND CURRENT_DATE()
          AND DATE(tc.coupon_created_at) >= DATE_ADD(CURRENT_DATE(),INTERVAL -395 DAY)
    GROUP BY 1,2,3,4,5),
    creation_table AS (
    SELECT cn.country_name AS country,
           IFNULL(u.city,'-') AS city,
           tc.talon_campaign_id AS campaign_id,
           COUNT(DISTINCT CASE WHEN tc.coupon_usage_limit = 1 THEN tc.coupon_id ELSE NULL END) AS created1,
           SUM(CASE WHEN tc.coupon_usage_limit > 1 THEN tc.coupon_usage_limit ELSE 0 END) AS created2
    FROM `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON tc.coupon_country_id = cn.country_id
    LEFT JOIN `peya-argentina.user_santiago_curat.city_usuarios` AS u ON tc.user_id = u.user AND cn.country_name = u.country
    WHERE DATE(tc.coupon_created_at) >= DATE_ADD(CURRENT_DATE(),INTERVAL -395 DAY)
          AND (DATE(tc.coupon_end) >= DATE_ADD(CURRENT_DATE(),INTERVAL -30 DAY) OR tc.coupon_start IS NULL)
    GROUP BY 1,2,3),
    coupons_final AS (
    SELECT ct.country AS country,
           ct.city AS city,
           ct.campaign_id AS campaign_id,
           SUM(ct.redeemed) AS redeemed,
           SUM(ct.redeemed_confirmed) AS redeemed_confirmed,
           SUM(ct.acquisitions) AS acq,
           --SUM(ct.used_amount) AS used_amount,
           --SUM(ct.used_amount_shopper) AS used_amount_shopper,
           --SUM(ct.used_amount_usd) AS used_amount_usd,
           --SUM(ct.used_amount_usd_shopper) AS used_amount_usd_shopper,
           SUM(ct.used_amount_confirmed) AS used_amount_confirmed,
           SUM(ct.used_amount_confirmed_shopper) AS used_amount_confirmed_shopper,
           SUM(ct.used_amount_confirmed_usd) AS used_amount_confirmed_usd,
           SUM(ct.used_amount_confirmed_usd_shopper) AS used_amount_confirmed_usd_shopper
    FROM coupons_table AS ct
    GROUP BY 1,2,3),
    creation_final AS (
    SELECT ct.country AS country,
           ct.city AS city,
           ct.campaign_id AS campaign_id,
           SUM(ct.created1) + SUM(ct.created2) AS created
    FROM creation_table AS ct
    GROUP BY 1,2,3)
SELECT IFNULL(crf.country,cf.country) AS Country,
       IFNULL(crf.city,cf.city) AS City,
       IFNULL(tca.talon_campaign_name,'-') AS Campaign,
       SUM(crf.created) AS Created,
       SUM(cf.redeemed) AS Redeemed,
       SUM(cf.redeemed_confirmed) AS Redeemed_Confirmed,
       SUM(cf.acq) AS Acq,
       --SUM(cf.used_amount) AS Used_Amount,
       --SUM(cf.used_amount_shopper) AS Used_Amount_Shopper,
       --SUM(cf.used_amount_usd) AS Used_Amount_USD,
       --SUM(cf.used_amount_usd_shopper) AS Used_Amount_USD_Shopper,
       SUM(cf.used_amount_confirmed) AS Used_Amount_Confirmed,
       SUM(cf.used_amount_confirmed_shopper) AS Used_Amount_Confirmed_Shopper,
       SUM(cf.used_amount_confirmed_usd) AS Used_Amount_Confirmed_USD,
       SUM(cf.used_amount_confirmed_usd_shopper) AS Used_Amount_Confirmed_USD_Shopper
FROM creation_final AS crf
FULL OUTER JOIN coupons_final AS cf ON crf.country = cf.country 
                                       AND crf.city = cf.city
                                       AND crf.campaign_id = cf.campaign_id
LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON crf.campaign_id = tca.talon_campaign_id OR cf.campaign_id = tca.talon_campaign_id
GROUP BY 1,2,3'''

# 2GB
q_fl = '''SELECT DISTINCT cn.country_name AS Country,
       IFNULL(u.city,'-') AS City,
       tca.talon_campaign_name AS Campaign
FROM `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON tc.coupon_country_id = cn.country_id
LEFT JOIN `peya-argentina.user_santiago_curat.city_usuarios` AS u ON tc.user_id = u.user AND cn.country_name = u.country
LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON tc.talon_campaign_id = tca.talon_campaign_id
WHERE DATE(tc.coupon_created_at) >= DATE_ADD(CURRENT_DATE(),INTERVAL -3 DAY)'''

# 23GB
q_st = '''WITH coupons_table AS (
    SELECT tc.order_id AS order_id,
           tc.user_id AS user_id,
           tca.talon_campaign_name AS campaign,
           tc.coupon_used_amount AS amount_used
    FROM `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc
    LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON tc.talon_campaign_id = tca.talon_campaign_id
    WHERE tc.order_id IS NOT NULL
          AND DATE(tc.order_registered_date) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -4 MONTH),MONTH) AND CURRENT_DATE()),
    orders_table AS (
    SELECT o.user.id AS user,
           cn.country_name AS country,
           IFNULL(c.city_name,'-') AS city,
           o.order_status AS status,
           o.order_id AS order_id,
           o.registered_date AS fecha,
           CASE WHEN ct.campaign IS NULL THEN 0 ELSE 1 END AS crm,
           IFNULL(o.shipping_amount_no_discount + o.amount_no_discount,0) AS total_paid,
           SUM(od.discount_amount) AS total_disc,
           SUM(CASE WHEN UPPER(od.discount_type_name) = 'VOUCHER' THEN od.discount_amount ELSE 0 END) AS total_voucher_disc
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o,
    UNNEST(discounts) AS od
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
    LEFT JOIN coupons_table AS ct ON o.order_id = ct.order_id
    WHERE o.registered_date BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -4 MONTH),MONTH) AND CURRENT_DATE()
    GROUP BY 1,2,3,4,5,6,7,8),
    acq_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           ot.city AS city,
           ot.fecha AS fecha,
           ct.campaign AS campaign,
           SUM(ct.amount_used) AS amount_used
    FROM orders_table AS ot
    INNER JOIN coupons_table AS ct ON ot.order_id = ct.order_id
    GROUP BY 1,2,3,4,5),
    cohorts_table AS (
    SELECT acq.user AS user,
           acq.country AS country,
           acq.campaign AS campaign,
           acq.city AS city,
           acq.fecha AS fecha,
           acq.amount_used AS original_amount_used,
           CASE WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) = 0 THEN 'ORIGINALS'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 1 AND 30 THEN 'M1'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 31 AND 60 THEN 'M2'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 61 AND 90 THEN 'M3' END AS cohort,
           IFNULL(COUNT(DISTINCT CASE WHEN ot.status != 'CONFIRMED' THEN NULL ELSE ot.order_id END),0) AS orders,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_paid END) AS total_paid,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_disc END) AS total_disc,
           SUM(CASE WHEN ot.status != 'CONFIRMED' THEN 0 ELSE ot.total_voucher_disc END) AS total_voucher_disc
    FROM acq_table AS acq
    LEFT JOIN orders_table AS ot ON acq.user = ot.user
    WHERE ot.fecha BETWEEN acq.fecha AND CURRENT_DATE()
    GROUP BY 1,2,3,4,5,6,7)
SELECT FORMAT_DATE('%Y-%m', ct.fecha) AS Month,
       ct.country AS Country,
       ct.city AS City,
       ct.campaign AS Campaign,
       SUM(ct.original_amount_used) AS Original_Amount_Used,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'ORIGINALS' THEN ct.user ELSE NULL END) AS Acquisitions,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M1' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M1_Users,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.orders ELSE 0 END) AS M1_Orders,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M2' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M2_Users,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.orders ELSE 0 END) AS M2_Orders,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M3' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M3_Users,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.orders ELSE 0 END) AS M3_Orders,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_paid ELSE 0 END) AS M1_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_disc ELSE 0 END) AS M1_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.total_voucher_disc ELSE 0 END) AS M1_Total_Voucher_Disc,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_paid ELSE 0 END) AS M2_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_disc ELSE 0 END) AS M2_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.total_voucher_disc ELSE 0 END) AS M2_Total_Voucher_Disc,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_paid ELSE 0 END) AS M3_Total_Paid,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_disc ELSE 0 END) AS M3_Total_Disc,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.total_voucher_disc ELSE 0 END) AS M3_Total_Voucher_Disc
FROM cohorts_table AS ct
GROUP BY 1,2,3,4'''

In [7]:
# Descargo la data
bq_tc = pd.io.gbq.read_gbq(q_tc, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 59258/59258 [00:12<00:00, 4790.33rows/s]


In [8]:
bq_fl = pd.io.gbq.read_gbq(q_fl, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████████████████████████████████████████████████████| 12717/12717 [00:01<00:00, 12637.43rows/s]


In [9]:
bq_st = pd.io.gbq.read_gbq(q_st, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████| 109782/109782 [00:45<00:00, 2431.57rows/s]


In [14]:
# Copio las bases
tc = bq_tc.copy()
fl = bq_fl.copy()
st = bq_st.copy()

### TRABAJO

#### TRABAJO COUPONS

In [15]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','City']
cols_float = [i for i in tc.columns if i not in cols_str]
tc[cols_float] = tc[cols_float].astype(float)
tc[cols_str] = tc[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
tc['Campaign'] = tc['Campaign'].str.replace(' ', '')

In [16]:
# Creo las columnas para los segmentos
tc['Benefit'] = tc['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
tc['Type'] = tc['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
tc['Segment'] = tc['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
tc['Level'] = tc['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
tc['Campaña'] = tc['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
tc['Automated'] = tc['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
tc['Budget'] = tc['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
tc['Trial'] = tc['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
# Coloco el IVA
tc = func_iva(tc)
# Marco las campañas a filtrar
tc['Filtrar'] = tc['Campaign'].apply(filtrar_cam)
tc = tc[tc['Filtrar'] == 'No'].copy()

In [17]:
# Saco el IVA
used_usd = 'Used_Amount_Confirmed_USD'
used_shopper_usd = 'Used_Amount_Confirmed_USD_Shopper'
used = 'Used_Amount_Confirmed'
used_shopper = 'Used_Amount_Confirmed_Shopper'
tc['Used_Final_USD'] = tc[used_shopper_usd] / (1 + tc['IVA']) * tc['Extra'] + tc[used_usd]
tc['Used_Final'] = tc[used_shopper] / (1 + tc['IVA']) * tc['Extra'] + tc[used]

#### TRABAJO FLOWS

In [18]:
# Doy formato a las columnas
fl = fl.apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
fl['Campaign'] = fl['Campaign'].str.replace(' ', '')
# Creo las columnas para los segmentos
fl['Benefit'] = fl['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
fl['Type'] = fl['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
fl['Segment'] = fl['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
fl['Level'] = fl['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
fl['Campaña'] = fl['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
fl['Automated'] = fl['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
fl['Budget'] = fl['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
fl['Trial'] = fl['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
# Marco las campañas a filtrar
fl['Filtrar'] = fl['Campaign'].apply(filtrar_cam)
fl = fl[fl['Filtrar'] == 'No'].copy()
# Divido los DF por Country
fl_arg = fl[fl['Country'] == 'ARGENTINA'].copy()
fl_chi = fl[fl['Country'] == 'CHILE'].copy()

#### TRABAJO STATS

In [19]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','Month','City']
cols_float = [i for i in st.columns if i not in cols_str]
st[cols_float] = st[cols_float].astype(float)
st[cols_str] = st[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
st['Campaign'] = st['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
st['Month'] = pd.to_datetime(st['Month'], format='%Y-%m').dt.strftime('%Y-%m')
# Ordeno la base
st.sort_values(by=['Country','Month','Campaign','City'],inplace=True)

In [20]:
# Llevo a cabo la limpieza del Reorder
cols_final = ['M1_Users_Final','M2_Users_Final','M3_Users_Final',
              'M1_Orders_Final','M2_Orders_Final','M3_Orders_Final',
              'Acq_M1','Acq_M2','Acq_M3']

for i in cols_final:
    st[i] = 0
# Completo las columnas que cree
cols_toFill_m1 = ['M1_Users_Final','M1_Orders_Final','Acq_M1']
cols_fill_m1 = ['M1_Users','M1_Orders','Acquisitions']
cols_toFill_m2 = ['M2_Users_Final','M2_Orders_Final','Acq_M2']
cols_fill_m2 = ['M2_Users','M2_Orders','Acquisitions']
cols_toFill_m3 = ['M3_Users_Final','M3_Orders_Final','Acq_M3']
cols_fill_m3 = ['M3_Users','M3_Orders','Acquisitions']
months = st.sort_values(by=['Month'],ascending=False)['Month'].unique().tolist()
if len(months) > 3:
    # Para todos los meses coloco los valores en M1
    st[cols_toFill_m1] = st[cols_fill_m1].values
    # Para el tercer y cuarto mes coloco los valores en M2
    st.loc[st['Month'].isin(months[2:]),cols_toFill_m2] = st[st['Month'].isin(months[2:])][cols_fill_m2].values
    # Para el cuarto mes coloco los valores en M3
    st.loc[st['Month'] == months[-1],cols_toFill_m3] = st[st['Month'] == months[-1]][cols_fill_m3].values
else:
    # Para todos los meses coloco los valores en M1
    st[cols_toFill_m1] = st[cols_fill_m1].values
    # Para el segundo mes coloco los valores en M2
    st.loc[st['Month'].isin(months[1:]),cols_toFill_m2] = st[st['Month'].isin(months[1:])][cols_fill_m2].values
    # Para el cuarto mes coloco los valores en M3
    st.loc[st['Month'] == months[-1],cols_toFill_m3] = st[st['Month'] == months[-1]][cols_fill_m3].values

In [21]:
# Hago una PT sacando el Month
index = ['Country','City','Campaign']
values = [i for i in st.columns if i not in index+['Month']]
pt_st = st.pivot_table(index=index,values=values,aggfunc='sum').reset_index()

In [22]:
# Creo las columnas para los segmentos
pt_st['Benefit'] = pt_st['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
pt_st['Type'] = pt_st['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
pt_st['Segment'] = pt_st['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
pt_st['Level'] = pt_st['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
pt_st['Campaña'] = pt_st['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
pt_st['Automated'] = pt_st['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
pt_st['Budget'] = pt_st['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
pt_st['Trial'] = pt_st['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
# Marco las campañas a filtrar
pt_st['Filtrar'] = pt_st['Campaign'].apply(filtrar_cam)
pt_st = pt_st[pt_st['Filtrar'] == 'No'].copy()

#### UNION COUPONS Y STATS

In [23]:
# Merge entre TC y Stats
on = ['Country','City','Campaign']
cols = ['Country','City','Campaign','Acq_M1','Acq_M2','Acq_M3','Acquisitions','M1_Orders','M1_Orders_Final','M1_Total_Disc',
        'M1_Total_Paid','M1_Total_Voucher_Disc','M1_Users','M1_Users_Final','M2_Orders','M2_Orders_Final','M2_Total_Disc','M2_Total_Paid',
        'M2_Total_Voucher_Disc','M2_Users','M2_Users_Final','M3_Orders','M3_Orders_Final','M3_Total_Disc','M3_Total_Paid',
        'M3_Total_Voucher_Disc','M3_Users','M3_Users_Final','Original_Amount_Used']
final = tc.merge(pt_st[cols],on=on,how='left').copy()
final.replace([np.nan,np.inf,-np.inf],'-',inplace=True)

In [25]:
# Ordeno las columnas
cols = ['Campaign','Country','City','Type','Segment','Level','Campaña','Automated','Budget','Trial']
cols_resto = [i for i in final.columns if i not in cols]
final = final[cols+cols_resto].copy()

In [26]:
# Divido por pais
cols_sort = ['City','Campaign']
final_arg = final[final['Country'] == 'ARGENTINA'].sort_values(by=cols_sort,ascending=False).copy()
final_chi = final[final['Country'] == 'CHILE'].sort_values(by=cols_sort,ascending=False).copy()
# Dropeo la columna country para achicar el DF
final_arg.drop(['Country'],axis=1,inplace=True)
final_chi.drop(['Country'],axis=1,inplace=True)

#### TIPOS DE CAMPAÑAS

In [38]:
# Creo un DF con las unique combinations
cols = ['Budget','Type','Segment','Trial','Level','Campaña','Automated','Country']
campaign_type = final.groupby(cols).size().reset_index().drop([0], axis = 1)
# Ordeno el DF
cols = ['Budget','Type','Segment']
campaign_type.sort_values(by=cols,ascending=False,inplace=True)
# Divido por pais
ct_arg = campaign_type[campaign_type['Country'] == 'ARGENTINA'].copy()
ct_chi = campaign_type[campaign_type['Country'] == 'CHILE'].copy()

### CARGA

In [40]:
# Carga Argentina
#carga(final_arg,'1_hOX88CV2BwFCPw37PH20116liF8nFdS4iEXLDL3LHM','Crudo Performance')
carga(ct_arg,'1_hOX88CV2BwFCPw37PH20116liF8nFdS4iEXLDL3LHM','Filtro Campaigns')
#carga(fl_arg,'1_hOX88CV2BwFCPw37PH20116liF8nFdS4iEXLDL3LHM','Crudo Active')

Carga Correcta!


In [42]:
# Carga Chile
carga(final_chi,'1CuWUY3WQhXHsZvpk7QMCEq9GJhSZU487as9_tFSBYF4','Crudo Performance')
#carga(ct_chi,'1CuWUY3WQhXHsZvpk7QMCEq9GJhSZU487as9_tFSBYF4','Filtro Campaigns')
#carga(fl_chi,'1CuWUY3WQhXHsZvpk7QMCEq9GJhSZU487as9_tFSBYF4','Crudo Active')

Carga Correcta!
